# WordEmbeddings

Ziele der heutigen Übung:
- WordEmbeddins anwenden
- Semantic Similarity berechnen können

Ein guter Artikel für diese Übung ist: https://www.geeksforgeeks.org/python-word-embedding-using-word2vec/  Lese ihn durch und wende ihn für diese Übung an.

Installiere als erstes die beiden Bibliotheken ```nltk```und ```gensim```

In [ ]:
pip install nltk

In [ ]:
pip install gensim

Wir möchten in der heutigen Übung ein Modell von WordEmbeddings auf Basis eines eigenen Textes generieren. Der Text sollte eine gewisse Länge haben, ein Buch ist nicht verkehrt. Freie Bücher findet man zum Beispiel auf https://www.gutenberg.org/

Lade den Text aus dem entsprechenden File:

In [1]:
BOOK = "../data/alice_im_wunderland.txt"

text = ""
with open(BOOK, encoding='utf8') as file:
    text = file.read()

Preprocessing und explorative Datenanalyse ist nie verkehrt, denn auf Basis eines schlechten, ungepflegten Datensatzes kann kein gutes Modell trainiert werden. Ersetze auf jeden Fall Zeilenumbrüche durch Leerzeichen. Je nach Buch sind eventuell auch weitere Anpassungen notwendig. Du solltest danach einen fortlaufenden Text haben. Es empfiehlt sich auch einige Statistiken über den Text auszugeben, um ein Gefühl für dessen Eigenschaften zu bekommen (z.B. Satz-, Wort-, Characteranzahl).Die NLTK Bibliothek hat dafür  einige praktischen Funktionen, die wir nun nutzen möchten. Recherchiere hierzu in der API, welche Funktionen sinnvoll sind:: https://www.nltk.org/api/nltk.tokenize.html

In [6]:
from nltk.tokenize import word_tokenize, sent_tokenize
from nltk.stem import WordNetLemmatizer

sentences = sent_tokenize(text, language='german')
words = word_tokenize(text, language='german')

print("Dataset Exploration:")
print("\tNumber of characters: ", len(text))
print("\tNumber of words: ", len(words))
print("\tNumber of sentences: ", len(sentences))

Dataset Exploration:
	Number of characters:  179155
	Number of words:  36303
	Number of sentences:  1196


Nutze nun ```gensim.models.Word2Vec``` um ein CBOW und ein SkipGram WordEmbedding-Modell aus deinem Text zu erstellen:

In [7]:
data = []

for i in sent_tokenize(text, language='german'):
    data.append([x.lower() for x in words])

In [29]:
from gensim.models import Word2Vec

VECTOR_SIZE = 100
WINDOW = 3

# Create CBOW model
model_cbow = Word2Vec(data, min_count=1, vector_size=VECTOR_SIZE, window=WINDOW)
model_cbow.name = "CBOW"

# Create SkipGram model
model_skip_gram = Word2Vec(data, min_count=1, vector_size=VECTOR_SIZE, window=WINDOW)
model_skip_gram.name = "SkipGram"

Schau dir bei beiden Modellen einige WordEmbeddings an.

In [30]:
for w in ['alice', 'wunderland', 'kaninchen', 'haus']:
    print("Word Embedding for: ", w)
    print(f"\t{model_cbow.name}: ", model_cbow.wv[w])
    print(f"\t{model_skip_gram.name}: ", model_skip_gram.wv[w], "\n")

Word Embedding for:  alice
	CBOW:  [-1.508644   -1.4417052   2.2125654  -3.6542668   1.1752645  -3.4046993
 -0.8015332   2.4277296  -1.2990925  -3.6446226  -3.546769   -0.84109956
 -1.2387382  -1.1329306   2.27871    -2.4218864  -2.709769   -1.0640365
  1.8315328  -1.4546776   1.62603    -1.2125131   2.6834517  -3.0582795
  0.43251997  2.1715367  -2.7675838  -0.11855579  0.62176126 -0.32994175
  0.31455478  1.8492532   2.6200619  -3.37198    -1.722747    0.19361149
 -4.1181393  -0.25668055  0.92288685 -6.2770896  -1.748529    0.11668386
  0.01992221  1.6744705   1.2690009  -4.800357   -3.0380406  -3.1093757
  2.0381167  -0.315546    2.6164594  -3.7813854   1.8640152   0.85524446
 -0.3313397  -2.0673804  -1.5127361   1.5595939  -0.05787027  1.135287
  0.01411045 -0.49057788 -1.5209627  -3.529223   -0.98461956  2.3181045
  3.963277    3.309091   -2.337658    4.754774   -3.4429991   2.1712663
 -1.3652229   2.7782166   0.6447172   1.8988823   0.14896776 -3.4390404
 -1.7776016   0.9475306  

In der Vorlesung haben wir die Ähnlichkeit von Wörtern mit der Kosinus-Ähnlichkeit zwischen zwei WordEmbedding Vektoren berechnet. Nutze die Formel um zwischen zwei Vektoren die Ähnlichkeit zu bestimmen:

In [31]:
from numpy import dot
from numpy.linalg import norm

sim_tuple = (('alice', 'wunderland'), ('alice', 'kaninchen'), ('alice', 'haus'))

for tuple in sim_tuple:
    for model in [model_cbow, model_skip_gram]:
        vec1 = model.wv[tuple[0]]
        vec2 = model.wv[tuple[1]]
        simliarity = dot(vec1, vec2) / (norm(vec1) * norm(vec2))
        print(f"Simliarity between {tuple[0]} and {tuple[1]} ({model.name}): {simliarity}")
    

Simliarity between alice and wunderland (CBOW): 0.13673104345798492
Simliarity between alice and wunderland (SkipGram): 0.09977753460407257
Simliarity between alice and kaninchen (CBOW): 0.10536202788352966
Simliarity between alice and kaninchen (SkipGram): 0.11443577706813812
Simliarity between alice and haus (CBOW): -0.1560482233762741
Simliarity between alice and haus (SkipGram): -0.14914971590042114


Nutze nun die ```similarity``` Funktion, ist das Ergebnis gleich deiner eigenen Berechnung? Woran könnte der Unterschied liegen?

In [32]:
for tuple in sim_tuple:
    for model in [model_cbow, model_skip_gram]:
        simliarity = model.wv.similarity(tuple[0], tuple[1])
        print(f"Simliarity between {tuple[0]} and {tuple[1]} ({model.name}): {simliarity}")

Simliarity between alice and wunderland (CBOW): 0.13673105835914612
Simliarity between alice and wunderland (SkipGram): 0.09977751970291138
Simliarity between alice and kaninchen (CBOW): 0.10536204278469086
Simliarity between alice and kaninchen (SkipGram): 0.11443576961755753
Simliarity between alice and haus (CBOW): -0.15604820847511292
Simliarity between alice and haus (SkipGram): -0.14914970099925995


Interessant ist die Funktion ```most_similar``` lese dir die Doku durch und probiere einige Wörter deines Buches aus:

In [33]:
model.wv.most_similar(positive=['alice', 'wunderland', 'kaninchen'], negative=['mann'])

[('zusammen', 0.3532821834087372),
 ('hastig', 0.3434976637363434),
 ('niederbrennen', 0.34009143710136414),
 ('glücklicherweise', 0.3340270519256592),
 ('einmaleins', 0.3179190158843994),
 ('mittel', 0.31787824630737305),
 ('etwas', 0.3178246021270752),
 ('wolltest.', 0.31777095794677734),
 ('sie', 0.3145189881324768),
 ('der', 0.31451767683029175)]